In [3]:
!pip install pandas
!pip install tensorflow
# !pip install transformers==4.30
print('test1')
!pip3 install -U scikit-learn scipy matplotlib
print('test2')
# !pip install keras==2.15.0
# !pip install tensorflow==2.15.0
# !pip install keras==2.15.0
!pip install xlrd
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install tensorflow-gpu

import xlrd
import pandas as pd
import tensorflow as tf
from tensorflow import keras
print('test3')
from sklearn.model_selection import train_test_split
print('test4')
import transformers
print('test5')
import keras
print('keras version',keras.__version__)
from transformers import TFBertForSequenceClassification, BertTokenizer,DataCollatorWithPadding
print('test6')
from sklearn.preprocessing import LabelEncoder
print('hello')
from keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from transformers import TFAutoModelForSequenceClassification

if tf.test.is_gpu_available():
    print("TensorFlow is running on GPU")
else:
    print("TensorFlow is running on CPU")
    
print(transformers.__version__)

# pip install accelerate
# pip install datasets

# import accelerate



#data = files.upload()

df = pd.read_excel('MCL LOINC Values.xls', sheet_name='Test Order LOINC',header=0)

label_encoder = LabelEncoder()

df['Encoded_LOINC'] = label_encoder.fit_transform(df['Order LOINC Code'])

df['Method Name'] = df['Method Name'].fillna('not entered')

df[['Test Order Name', 'Method Name']] = df[['Test Order Name', 'Method Name']].apply(lambda x: x.str.lower())



import tensorflow as tf


def preprocess_data(df):
    tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

    inputs = tokenizer(df[['Test Order Name','Method Name']].values.tolist(),padding=True, truncation=True, return_tensors="tf")
    #start of commented code
#     test_order_name_tokens = tokenizer(df['Test Order Name'].tolist(), padding=True, truncation=True, return_tensors="tf")

#     method_names = [method_name for method_name in df['Method Name'].tolist() if method_name]
#     method_name_tokens = tokenizer(method_names, padding=True, truncation=True, return_tensors="tf")

#     input_ids = tf.concat([test_order_name_tokens['input_ids'], method_name_tokens['input_ids']], axis=1)
#     attention_mask = tf.concat([test_order_name_tokens['attention_mask'], method_name_tokens['attention_mask']], axis=1)

# # Create input dictionary
#     inputs = {
#         'input_ids': input_ids,
#         'attention_mask': attention_mask,
#     }

    #End of Commented code
    data_collator = DataCollatorWithPadding(tokenizer)
    return inputs, df['Encoded_LOINC'],data_collator,tokenizer

# Define fine-tuning function
#def fine_tune_model(train_inputs, train_labels):
def fine_tune_model(train_dataset, eval_dataset, data_collator,tokenizer):


    from transformers import Trainer
    from transformers import TFTrainingArguments
    #model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels = 2494)

    # strategy = tf.distribute.MirroredStrategy()

    # training_args = TFTrainingArguments(
    # per_device_train_batch_size = 16,
    # per_device_eval_batch_size=16,
    # num_train_epochs=5,
    # learning_rate = 2e-5,
    # weight_decay = 0.01,
    # output_dir='./logs',

    # )

    #with strategy.scope():


    model = TFBertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=2494, from_pt=True)# loading weights from pytorch as weights from tensor flow not available

      # from transformers import TFTrainer

      # trainer = TFTrainer(
      # model=model,
      # args=training_args,
      # train_dataset = train_dataset,
      # eval_dataset = eval_dataset,
      # #data_collator = data_collator,
      # #tokenizer = tokenizer

      # )
      # trainer.train()

      # predictions = trainer.predict(eval_dataset)
      # print(predictions.prediction.shape,predictions.label_ids.shape)

      # from datasets import load_metric

      # metric = load_metric("glue","mrpc")
      # preds = np.argmax(predictions.predictions,axis=1)
      # metric.compute(predictions=preds, references=predictions.label_ids)


    # Define optimizer and compile the model

    #optimizer = tf.keras.optimizers.adam(learning_rate=1e-5)
   # optimizer1 = tf.keras.optimizers.Adam(learning_rate=1e-5)

   #Second comment starts here

    per_device_train_batch_size = 16
    per_device_eval_batch_size = 16
    num_train_epochs = 6
    learning_rate = 2e-5
    #weight_decay = 0.01
    output_dir = './logs'

    # # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')

    # # Compile the model with the defined optimizer and loss function
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    # # Train the model
    # history = model.fit(
    #     train_dataset,  # Your training dataset
    #     epochs=num_train_epochs,
    #     batch_size=per_device_train_batch_size,
    #     validation_data=eval_dataset,  # Your evaluation dataset
    #     validation_batch_size=per_device_eval_batch_size,
    #     callbacks=[],  # Any callbacks you want to use during training
    #     verbose=1  # Set verbosity level (0 = silent, 1 = progress bar, 2 = one line per epoch)
    # )

    #Second comment ends here

    #model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy())




    # Train the model
    model.fit(train_inputs, train_labels, epochs=num_train_epochs)

    return model
    #return history



def evaluate_model(model, test_inputs, test_labels):

    predictions = model.predict(test_inputs)


    logits = predictions.logits
    predicted_classes = tf.argmax(logits, axis=1)

    # Convert TensorFlow tensors to numpy arrays
    test_labels_numpy = test_labels.values

    for i in range(len(predicted_classes)):
      print(f"Sample {i + 1}: Predicted Class = {predicted_classes[i]}, True Label = {test_labels_numpy[i]}")

    # Calculate accuracy
    accuracy = accuracy_score(test_labels_numpy, predicted_classes)

    return accuracy

#Step1
inputs, labels, data_collator,tokenizer = preprocess_data(df)

input_ids = inputs['input_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

#Step2

#train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)
#train_inputs, eval_inputs, train_labels, eval_labels = train_test_split(inputs, labels, test_size=0.2, random_state=42)
train_input_ids, eval_input_ids, train_attention_mask, eval_attention_mask, train_labels, eval_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_mask}
eval_inputs = {'input_ids': eval_input_ids, 'attention_mask': eval_attention_mask}

# Create TensorFlow datasets for training and evaluation
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
eval_dataset = tf.data.Dataset.from_tensor_slices((eval_inputs, eval_labels))

batch_size = 40

# Batch and shuffle train dataset
train_dataset = train_dataset.shuffle(buffer_size=len(train_input_ids)).batch(batch_size)

# Batch eval dataset
eval_dataset = eval_dataset.batch(batch_size)

#Step3
# model = fine_tune_model([train_input_ids, train_attention_mask], train_labels)
model = fine_tune_model(train_dataset, eval_dataset, data_collator,tokenizer)

#Step4
# Evaluate the fine-tuned model
accuracy = evaluate_model(model, [eval_input_ids, eval_attention_mask], eval_labels)

# # Print the classification metrics
print("Accuracy:", accuracy)

test1
test2
Looking in indexes: https://download.pytorch.org/whl/cu118
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-7gwei9by/tensorflow-gpu_30579693a59740a18b3ac0ff2ec3f0d2/setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a proble

2024-04-08 08:31:56.766826: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
41/87 [=============>................] - ETA: 23:22 - loss: 14.4864 - accuracy: 0.0015

KeyboardInterrupt: 

In [ ]:
''